# Computational Social Science Project #3 

*Group number:* 5

*Group members:* Nadia Almasalkhi, Daniel Lobo, Luyi Jian

*Semester:* Fall 2021

# 1. Libraries setup

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelBinarizer


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

sns.set_style("darkgrid")
np.random.seed(273)

# Make sure to import other libraries that will be necessary for training models!

# 2. Data Pre-Processing and Cleaning

In [ ]:
# Inspections Data 2011 - 2013
chicago_inspections_2011_to_2013 = pd.read_csv("data/Chicago Inspections 2011-2013.csv", 
                                              low_memory=False)

# Inspections Data 2014
chicago_inspections_2014 = pd.read_csv("data/Chicago Inspections 2014.csv", 
                                      low_memory=False)

In [ ]:
# Look at the inspections data
chicago_inspections_2011_to_2013.head()

In [ ]:
# List column names
chicago_inspections_2011_to_2013.columns

In [ ]:
# Drop column names related to geography, identification, and pass/fail flags that perfectly predict the outcome
chicago_inspections_2011_to_2013.drop(columns = ['AKA_Name', 
                                                 'License',
                                                'Address',
                                                'City',
                                                'State',
                                                'Zip',
                                                'Latitude',
                                                'Longitude',
                                                'Location',
                                                'ID',
                                                'LICENSE_ID',
                                                 'LICENSE_TERM_START_DATE',
                                                 'LICENSE_TERM_EXPIRATION_DATE',
                                                 'LICENSE_STATUS',
                                                'ACCOUNT_NUMBER',
                                                'LEGAL_NAME',
                                                'DOING_BUSINESS_AS_NAME',
                                                'ADDRESS',
                                                'CITY',
                                                'STATE',
                                                'ZIP_CODE',
                                                'WARD',
                                                'PRECINCT',
                                                'LICENSE_CODE',
                                                'BUSINESS_ACTIVITY_ID',
                                                'BUSINESS_ACTIVITY',
                                                'LICENSE_NUMBER',
                                                'LATITUDE',
                                                'LONGITUDE',
                                                'pass_flag',
                                                'fail_flag'],
                                     inplace = True)

chicago_inspections_2011_to_2013.set_index(['Inspection_ID', 'DBA_Name'], inplace = True)

In [ ]:
# Convert the Inspection Date to a datetime format
chicago_inspections_2011_to_2013['Inspection_Date'] = pd.to_datetime(chicago_inspections_2011_to_2013['Inspection_Date'], infer_datetime_format=True)  

In [ ]:
chicago_inspections_2011_to_2013.head()

## Visualization

What do inspections look like over time?

In [ ]:
# Visualize Inspections Over Time
chicago_inspections_2011_to_2013['Inspection_MonthYear'] = chicago_inspections_2011_to_2013['Inspection_Date'].dt.to_period('M')
counts_by_day = chicago_inspections_2011_to_2013.groupby('Inspection_MonthYear').count().rename(columns = {'Facility_Type': 'Count'})['Count'].reset_index()
counts_by_day.set_index(["Inspection_MonthYear"], inplace = True)
counts_by_day.plot(title = "Inspections by Month and Year") 

What do the results look like? 

**Answer:** It looks like the volume of inspections vary over the year, falling into lulls for months at a time (particularly around the holidays in December and in the month of July). It also appears that there were more inspections in 2013 than in 2012.
<font color = 'blue'> The count of inspection in Jan. 2012 is over 500 and in Jan. 2013 is nearly 800, which seems to be not falling into lulls? -Luyi </font>

<font color = 'green'> The line graph plot shows that the number of inspections varies pretty dramatically over the course of the year. Overall, we see fewer inspections in 2012 compared to the first three quarters of 2013. There also appears to be a trend of inspections rising in the winter months and declining in the spring/early summer months. -Daniel </font>

In [ ]:
# Inspection Results
sns.catplot(data = chicago_inspections_2011_to_2013,
           x = "Results",
           kind = "count")

plt.title("Inspection Results")
plt.show()

What if we separate by facility type?

In [ ]:
# Inspection Results by Facility Type (Restaurant or Not)
sns.catplot(data = chicago_inspections_2011_to_2013,
           x = "Results",
           kind = "count",
           hue = 'Facility_Type_Clean')

plt.title("Inspection Results by Facility Type")
plt.show()

## Preprocess Data

In [ ]:
# Drop datetime info
chicago_inspections_2011_to_2013 = chicago_inspections_2011_to_2013.dropna().drop(['Inspection_Date',
                                      'minDate',
                                      'maxDate',
                                      'Inspection_MonthYear'],
                                      axis = 1)

In [ ]:
# Set target variable. 
y = chicago_inspections_2011_to_2013['Results']
## Comment out the following code if you don't want to binarize the target variable
y = y.replace({'Pass w/ Conditions': 'Pass'})
lb_style = LabelBinarizer()
y = lb_style.fit_transform(y)
# Recode 0s and 1s so 1s are "Fail"
y = np.where(y == 1, 0 ,1)

# All other features in X
X = chicago_inspections_2011_to_2013.drop(columns = ['Results'])


In [ ]:
X = pd.get_dummies(X)

In [ ]:
X.head()

It seems like we should merge `Facility_Type_1023`, because currently they appear as 4 slightly different columns:
* `Facility_Type_1023 CHILDERN'S SERVICE FACILITY`	
* `Facility_Type_1023 CHILDERN'S SERVICE S FACILITY`	
* `Facility_Type_1023 CHILDERN'S SERVICES FACILITY`	
* `Facility_Type_1023-CHILDREN'S SERVICES FACILITY`
* `Facility_Type_Children's Services Facility`

-*Nadia*

In [ ]:
# To merge `Facility_Type_1023` - Luyi 

X['Facility_Type_1023 CHILDERNS SERVICE FACILITY'] = X.apply(lambda x: x["Facility_Type_1023 CHILDERN'S SERVICE FACILITY"] + x["Facility_Type_1023 CHILDERN'S SERVICE S FACILITY"] + x["Facility_Type_1023 CHILDERN'S SERVICES FACILITY"] + x["Facility_Type_1023-CHILDREN'S SERVICES FACILITY"] + x["Facility_Type_Children's Services Facility"], axis=1)

X = X.drop(columns = ["Facility_Type_1023 CHILDERN'S SERVICE FACILITY",
                      "Facility_Type_1023 CHILDERN'S SERVICE S FACILITY",
                     "Facility_Type_1023 CHILDERN'S SERVICES FACILITY",
                     "Facility_Type_1023-CHILDREN'S SERVICES FACILITY",
                     "Facility_Type_Children's Services Facility"])

In [ ]:
X["Facility_Type_1023 CHILDERNS SERVICE FACILITY"] = X["Facility_Type_1023 CHILDERNS SERVICE FACILITY"].astype(int)

In [ ]:
X.head()

# 3. Fit Models

## 3.1 Data Splitting

In [ ]:
# Create a training set and a validation set. Per the instructions, we do NOT create a test set.
X_train, X_validate, y_train, y_validate = train_test_split(X, y, train_size = .80, test_size=0.20,
                                                   stratify=y)

## 3.2 Fit Models

### 3.2.1 Model 1 - Support Vector Machine

Detail the basic logic and assumptions underlying each model, its pros/cons, and why it is a plausible choice for this problem:

SVM: The maximal margin classifier, the support vector classifier, and the support vector machine are often referred as “support vector machines”. Specifically, Maximal Margin Classifier (AKA Hard Margin)uses a hyperplane that separates the training observations perfectly according to their class labels; while Support Vector Classifier / Soft Margin Classifier use the same logic of hyperplane but allow misclassifying a few training observations to get better predictions in testing observations (namely, to avoid overfitting in training set). 

In other words, Hard Margin finds the hyperplane and margin that correctly classifies all points. It is sensitive to single observations/outliers, has the risk of overfitting, and will fail entirely if the data are not linearly separable. But Soft Margin, by allowing some points to “violate” the margin, trades off some bias for better variance (avoiding overfitting).

For this problem, because there are so many features and we don't know if the data are linearly separable, it's better to use SVM to allow some fexibility in prediction. 


(FROM THE INSTRUCTION) Be sure to do the following:

1. Import the appropriate library from sklearn
2. Set up a hyperparameter grid (check out our previous labs to see how to do this)
3. Find the best hyperparameters, and then fit your model (using train/validation splits or cross-validation)

### Untuned SVM model

In [ ]:
# create a model
svm = SVC()

# fit the model
svm_model = svm.fit(X_train, y_train.ravel())

y_pred = svm_model.predict(X_validate)

In [ ]:
# look at the confusion matrix of SVM 
cf_matrix = confusion_matrix(y_validate, y_pred, normalize = "true")

df_cm = pd.DataFrame(cf_matrix, range(2),
                  range(2))

df_cm = df_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
df_cm.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#### Interpretation:
From the confusion matrix, the SVM predicts well for facilities that passed the inspection (1.0), but predicts poorly for those failed the inspection (0.06), due to the imbalance of data structure where very few facilities failed inspection. Next we will search for optimal hyperparameters and see if the SVM with tuned parameters will perform better. 

### Tuned SVM model

In [ ]:
### not able to get an output from this chunk of code, so I comment it out 

### use GridSearchCV to search for optimal hyperparameters  (Hyperparameter Tuning)

#import warnings
#from sklearn.exceptions import DataConversionWarning
#warnings.filterwarnings(action='ignore')
#from sklearn.metrics import accuracy_score

### Define the hyperparameters to be tested out
#svm_param_grid = {'C': [0.1, 1, 10],   # C: the regularization parameter, C, of the error term.
              #'gamma': [1, 0.1, 0.01],  # gamma: the kernel coefficient for ‘rbf’, ‘poly’, and ‘sigmoid’. If gamma is ‘auto’, then 1/n_features will be used instead.
              #'kernel': ['rbf']}  # kernel: specifies the kernel type to be used in the algorithm. It can be ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’, or a callable. The default value is ‘rbf’.

### Initiate grid search for optimal hyperparameters
#svm_grid = GridSearchCV(svm_model, svm_param_grid, cv=3)

### Fit gird search on training data and save it as a grid of results
#svm_grid_results = svm_grid.fit(X_train, y_train_rav)

### Get the best model (the one that works best on left-out obs in cross validation)
#best_model_svm = svm_grid_results.best_estimator_

### Get the parameters that make up the best estimator
#best_parameters = svm_grid_results.best_params_

### Get the best results (Mean cross-validated score of the best_estimator)
#best_results = svm_grid_results.best_score_

### Print the best parameters and the best mean test score 
#print("Best parameters are", best_parameters)
#print("Best parameters produce mean test score of", best_results)

In [ ]:
### Get best predictions by predicting on validation set
#best_svm_pred = best_model_svm.predict(X_validate)

### create a confusion matrix to evaluate the tuned svm model on the validation set
#cf_matrix_svm = confusion_matrix(y_validate_rav, best_svm_pred, normalize = "true")

#df_cm = pd.DataFrame(cf_matrix_svm, range(2),
                      #range(2))

#df_cm = df_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
#df_cm.index = ["Pass", "Fail"]
#plt.figure(figsize = (8,5))
#sns.set(font_scale=1.4)#for label size
#sns.heatmap(df_cm, 
           #annot=True,
           #annot_kws={"size": 16},
           #fmt='g')

#plt.title("Confusion Matrix for Tuned SVM Model on Validation Set")
#plt.xlabel("Predicted Label")
#plt.ylabel("True Label")
#plt.show()

### Support Vector Machine Validation Metrics
In this section, accuracy, precision, recall, and F1 score will be caculated based on the performance of untuned SVM model on the validation set.

In [ ]:
# Create the predictions from the untuned SVM model on the validation set.

y_validate_pred_svm = svm_model.predict(X_validate)

### Accuracy
A measure of the number of correction predictions regardless of direction, divided by the total number observations. Accuracy can be expressed as:

$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(y_validate_pred_svm)): 
    if y_validate[i]==y_validate_pred_svm[i]==1:
       TP += 1
    if y_validate_pred_svm[i]==1 and y_validate[i]!=y_validate_pred_svm[i]:
       FP += 1
    if y_validate[i]==y_validate_pred_svm[i]==0:
       TN += 1
    if y_validate_pred_svm[i]==0 and y_validate_pred_svm[i]!=y_validate[i]:
       FN += 1

accuracy = (TP + TN)/(TP + TN + FP + FN)
print("Accuracy of the untuned SVM model is", accuracy)

The untuned SVM model's accuracy (=0.853) is fair, but may not as good as the accuracy of Decision Tree classifier or Random Forest.

### Precision
Precision is a measure of how well calibrated predictions are. Specifically, it tells us of the predictions in the positive class ("failing inspection" in this case) we made, how many were correct. The formula for precision is:

$$
Precision = \frac{TP}{TP + FP}
$$

In [ ]:
precision = TP/(TP + FP)
print("Precision of the untuned SVM mdoel is", precision)

Output: Precision of the untuned SVM mdoel is 0.9158415841584159

### Recall
Recall is a measure that tells us, of all of the positive class members in the ground truth labels, how many did we successfully predict as positive? Recall is defined as:

$$
Recall = \frac{TP}{TP + FN}
$$

In [ ]:
recall = TP/(TP+FN)
print("Recall of the untuned SVM mdoel is", recall)

Output: Recall of the untuned SVM mdoel is 0.2993527508090615

This low value of recall is not surprising, given the untuned SVM model does a poor job predicting "Failed" cases (0.3).

### F1 Score
The F1 Score is defined as:

$$
F1 = 2 * \frac{precision * recall}{precision + recall}
$$


In [ ]:
f1 = 2 * (precision * recall)/(precision + recall)
print("F1 Score for the untuned SVM mdoel is", f1)

Output: F1 Score for the untuned SVM mdoel is 0.45121951219512196

### 3.2.2 Model 2 - Decision Tree Classifier

Decision Trees are are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation. 

Some advatanges of Decision Trees are the following: They are simple to understand, to interpret, and to visualize. They require little data preparation and are able to handle both numerical and categorical data. They are considered white box models instead of black box models, meaning that if a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. Black box models on the other hand, like neural networks, are more difficult to interpret. Decision trees are able to handle multi-output problems. It is possible to validate a Decision Tree model using statistical tests, making it possible to account for the reliability of the model. Lastly, Decision Trees perform well even if its assumptions are somewhat violated by the true model from which the data were generated.

Some disadvantages of Decision Trees are the following: Overly complex Decision Trees can overfit to the data. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem. Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble. Becuase Decision Tree predictions are piece-wise constant approximations and not continuous, they are not good at extrapolation. Practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.

The Decision Tree Classifier is a plausible choice for this problem because our data is non-linear and our outcome variable is not continuous. 

### Training the Model

In [ ]:
# Initialize a Decision Tree Classifier
dt_classifier = tree.DecisionTreeClassifier(criterion='gini',  # or 'entropy' for information gain
                       splitter='best',  # or 'random' for random best split
                       max_depth=None,  # how deep tree nodes can go
                       min_samples_split=2,  # samples needed to split node
                       min_samples_leaf=1,  # samples needed for a leaf
                       min_weight_fraction_leaf=0.0,  # weight of samples needed for a node
                       max_features=None,  # number of features to look for when splitting
                       max_leaf_nodes=None,  # max nodes
                       min_impurity_decrease=1e-07, #early stopping
                       random_state = 10) #random seed

In [ ]:
# Use cross validation to train the model on our data 
scores = cross_val_score(dt_classifier, X, y, cv=5)

In [ ]:
scores

In [ ]:
# Take the mean score from the results of cross validation
scores.mean()

#### Interpretation: 
We see that our Decision Tree Classifier model is about 90 percent accurate in predicting inspection results. 

### Feature Selection

A unique aspect of tree-based methods is feature importance. One way to calculate feature importance is to see how much information each new feature adds. If a feature does not add any or very little information to a prediction, it may be possible to safely drop it.

In [ ]:
dt_classifier.fit(X_train, y_train)
feat_importances = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(dt_classifier.feature_importances_))], axis = 1)
feat_importances.columns = ["Feature", "Importance"]
sns.barplot(x = "Importance", y = "Feature", data = feat_importances.nlargest(10, 'Importance'))
plt.show()

#### Interpretation: 
We see that seriousCount is by far the most important feature in the model, far more important than the the next 9 most important features combined. We may worry that this feature is biasing our results preductions given its outsized importance relative to the other most important features. Thus, we will try removing it from the model and see how our predictions are affected. 

In [ ]:
# Create a list with the top 9 features for the Decision Tree Classifier, excluding 'seriousCount'
dt_feature_df = feat_importances.nlargest(10, 'Importance')
dt_feature_list = list(dt_feature_df["Feature"])
dt_feature_list.remove('seriousCount')
dt_feature_list

In [ ]:
# Reduce dataset X to include only the select 10 features
reduced_X = X[dt_feature_list]
reduced_X.head()

In [ ]:
# split the reduced data into training and test set
reduced_X_train, reduced_X_validate, y_train, y_validate = train_test_split(reduced_X, y, train_size = .80, test_size=0.20,
                                                   stratify=y)

In [ ]:
# Re-fit the Decision Tree Classifier on reduced data
dt_classifier.fit(reduced_X_train, y_train)

In [ ]:
# Use cross validation to train the model on our reduced data 
scores = cross_val_score(dt_classifier, reduced_X, y, cv=5)

In [ ]:
scores

In [ ]:
scores.mean()

#### Interpretation: 
We see that dropping all but the 9 most important features of the model (exlcuding 'seriousCount') dramatically reduces the accuracy of our Decision Tree Classifier's predictions on the training set from 90 percent to 68 percent. Thus, we will keep our original full feature set for our DT model predictions moving forward.  

### Visualizing our Decision Tree Classifier

In [ ]:
# To visualize our Decision Tree Classifier model on the original data set
dt_classifier.fit(X_train, y_train)

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dt_classifier, 
                   feature_names=X.columns,  
                   class_names=["<=50k", ">50k"],
                   filled=True,
                  fontsize = 10,
                  max_depth = 4)

### Confusion Matrix

Now we can visualize the model predictions on the validation set using a Confusion Matrix. 

In [ ]:
y_pred = dt_classifier.predict(X_validate)

In [ ]:
cf_matrix = confusion_matrix(y_validate, y_pred, normalize = "true")

df_cm = pd.DataFrame(cf_matrix, range(2),
                  range(2))

df_cm = df_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
df_cm.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#### Interpretation: 
We see that our Decision Tree Classifier is a pretty strong model. It correctly predicts a passing inspection 94 percent of the time and correctly predicts a failing inspection 80 percent of the time. 

In [ ]:
ax = sns.distplot(y, kde = False)
ax.set_title("Distribution of Target Variable (Inspection Results)")
ax.set(xlabel='Variable Value', ylabel='Count')
plt.show()

#### Interpretation: 
By plotting our target variable, we can see that a class imbalance between passing inspections (coded as 0) and failing inspections (coded as 1) is likely driving the poorer predictions of failing inspections from our Decision Tree Classifier model. 

### Hyperparameter Tuning
To prevent overfitting of the Decision Tree Classifier model, one thing we could do is define some parameter which ends the recursive splitting process. This may be a parameter such as maximum tree depth or minimum number of samples required in a split. Controlling these model hyperparameters is the easiest way to counteract overfitting.

In [ ]:
# Import tools for evaluating Decision Tree Classifiers
from sklearn.metrics import accuracy_score, classification_report

# Define the hyperparameters to be tested out
# Options include the default and an alternate option
param_grid_dt = {
    'criterion': ['gini','entropy'],
    'max_depth': range(1,10),
    'min_samples_split': range(1,10),
    'min_samples_leaf': range(1,5),
}

In [ ]:
# Initiate grid search
dt_grid_search = GridSearchCV(dt_classifier, param_grid_dt, cv=3)

In [ ]:
# Fit grid search to training data and save it as a grid of results
dt_grid_results = dt_grid_search.fit(X_train, y_train)

In [ ]:
# Determine the best model (the one that works best on left-out obs in cross validation)
best_model_dt = dt_grid_results.best_estimator_

In [ ]:
# Explicate the parameters that make up the best estimator
best_parameters = dt_grid_results.best_params_

# Identify the best results (Mean cross-validated score of the best_estimator)
best_results = dt_grid_results.best_score_

# Let's see what we're working with
print("Best parameters are", best_parameters)
print("Best parameters produce mean test score of", best_results)

#### Interpretation: 
Tuning the hyperparameters of our Decision Tree Classifier model reduces its accuracy on our training set from 90 percent to 80 percent. This could indicate less hyperfitting to the training data set, which may be a good thing. Let's see how this model compares on predictions on the validation data set. 

### Confusion Matrix of Tuned Model 

In [ ]:
y_pred = best_model_dt.predict(X_validate)

In [ ]:
range(len(y_pred))

In [ ]:
cf_matrix = confusion_matrix(y_validate, y_pred, normalize = "true")

df_cm = pd.DataFrame(cf_matrix, range(2),
                  range(2))

df_cm = df_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
df_cm.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix of Tuned Model on Validation Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#### Interpretation: 
I'm not sure why this Confusion Matrix is coming out like this...I'm going to ask KQ tomorrow. 

### 3.3.2 Model 2 - Decision Tree Classifier Validation Metrics

### Accuracy
A measure of the number of correction predictions regardless of direction, divided by the total number observations. Accuracy can be expressed as:

$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

In [ ]:
range(len(y_pred))

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(y_pred)): 
    if y_validate[i]==y_pred[i]==1:
       TP += 1
    if y_pred[i]==1 and y_validate[i]!=y_pred[i]:
       FP += 1
    if y_validate[i]==y_pred[i]==0:
       TN += 1
    if y_pred[i]==0 and y_pred[i]!=y_validate[i]:
       FN += 1

In [ ]:
accuracy = (TP + TN)/(TP + TN + FP + FN)
print("Accuracy is", accuracy)

### Precision
Precision is a measure of how well calibrated predictions are. Specifically, it tells us of the predictions in the positive class ("failing inspection" in this case) we made, how many were correct. The formula for precision is:

$$
Precision = \frac{TP}{TP + FP}
$$

In [ ]:
precision = TP/(TP + FP)
print("Precision is", precision)

### Recall
Recall is a measure that tells us, of all of the positive class members in the ground truth labels, how many did we successfully predict as positive? Recall is defined as:

$$
Recall = \frac{TP}{TP + FN}
$$

In [ ]:
recall = TP/(TP + FN)
print("Recall is", recall)

### F1 Score
The F1 Score is defined as:

$$
F1 = 2 * \frac{precision * recall}{precision + recall}
$$


In [ ]:
f1 = 2 * (precision * recall)/(precision + recall)
print("F1 Score is", f1)

#### Interpretation: 
In our case of predicting whether a Chicago business passes or fails a food inspection, where a true positive (TP) is correctly precicting that a business fails inspection and a false positive (FP) is incorrectly predicting that a business fails inspection, FPs and false negatives (FN) are high stakes errors. We don't want to distrupt the operations of a business that passes inspection with a FP because that would be unfair to business owners and potentially detrimental to the community. We don't want to allow business that actually fails inspection to continue serving food to customers with a FN because that could be harmful to the health of the community. 

Thus, we may want to prioritize the precision measure of true negatives (TNs) in this case. This would tell us, of all the establishments that we predicted to pass inspection, what proportion did we correctly predict (TN/ FN + TN)? This is perhaps a more urgent measure to know than overall accuracy since it gives us confidence that our predictive model is causing minimal harm to people in the community. The precision measure of true positives (TPs), assuming it is high enough, would give us confidence that our predictive model is causing minimal harm to food-serving businesses in the community. 

### 3.2.3 - Model 3: Random Forest Classifier

The random forests method chooses which feature to use at a split from a random subsample of features. In a random forest, multiple decision trees (often 100) are created and predictions are made based on each decision tree in the forest's vote on how to classify a given case.

The random subset used in Random Forests allows it to sidestep the issue of Decision Trees being "greedy" or shortsighted. Random Forests also allow for an estimation of probabilities rather than just straightforward "decisions" about how to classify the case.

In [ ]:
# Inspect the shape of the y_train array
y_train.shape
y_validate.shape

`y_train` and `y_validate` are irregularly shaped and will thus need to use `.ravel()` in order to work well

In [ ]:
y_train_rav = y_train.ravel()
y_validate_rav = y_validate.ravel()

In [ ]:
# Initialize the Random Forest classifier, with hyperparameters that leave it relatively unrestricted.

rf_classifier = RandomForestClassifier(criterion='gini',  # same as default
                       max_depth=None,  # same as default
                       min_samples_split=2,  # same as default
                       min_samples_leaf=1,  # same as default
                       min_weight_fraction_leaf=0.0,  # same as default
                       max_features=None,  # number of features to look for when splitting
                       max_leaf_nodes=None,  # same as default
                       min_impurity_decrease=1e-07, #early stopping
                       random_state = 10) #random seed

In [ ]:
# Fit this model to the training data

rf_model = rf_classifier.fit(X_train, y_train_rav)

In [ ]:
# Now let's evaluate the accuracy of the rt_classifier on the training set

y_train_pred_rf = rf_model.predict(X_train) # here we create the prediction of the rf_model for the training set data

In [ ]:
# Let's display the accuracy of the RF model on the training set
rf_cf_matrix = confusion_matrix(y_train_rav, y_train_pred_rf, normalize = "true")

rf_cm = pd.DataFrame(rf_cf_matrix, range(2),
                  range(2))

rf_cm = rf_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
rf_cm.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4) #for label size
sns.heatmap(rf_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

The Random Forests Model has 99% accuracy on the training set, which makes sense because that's what the model was trained on and we allowed the trees within the forest to go down to leaves of *n* = 1. The real test now will be to see how it performs on the validation set. If it performs poorly on the validation set, then that is a sign of the model overfitting the training data, and we would need to go back and restrict the overfitting of the rf_classifier by limiting the maximum depth, raising the bar for the minimum impurity decrease, or other changes to the hyperparameters.

In [ ]:
# Calculate predictions for validation set using rf_model
y_validate_pred_rf = rf_model.predict(X_validate)

rf_cf_matrix = confusion_matrix(y_validate_rav, y_validate_pred_rf, normalize = "true")

rf_cm = pd.DataFrame(rf_cf_matrix, range(2),
                  range(2))

rf_cm = rf_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
rf_cm.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4) #for label size
sns.heatmap(rf_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix for Baseline RF Classifier Model on Validation Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

`rf_model` does alright in predicting on the validation set (Passes are accurately predicted 93% of the time; Fails are accurately predicted 89% of the time) but perhaps it could do better. Let us turn to **hyperparameter tuning** to see how changes there could improve the generalizability of the RF model.

In [ ]:
# Hyperparameter Tuning
# This is what was originally run to determine the optimal hyperparameters for X_train, y_train_rav
# But it takes a long time, so we are not going to run it again.
# Instead, we just show the code that we originally used, and hard-code the tuned model below.

# CODE WE ORIGINALLY USED

## Import tools for evaluating random forests
#from sklearn.metrics import accuracy_score, classification_report

## Define the hyperparameters to be tested out
## Options include the default and an alternate option.
#param_grid_rf = {
#    'n_estimators': [100, 150],
#    'max_depth': [10, None],
#    'max_features': [None, 'sqrt'],
#    'min_impurity_decrease': [0.0000001, 0.0],
#    'random_state': [10]
#}

## Initiate grid search
#rf_grid_search = GridSearchCV(rf_classifier, param_grid_rf, cv=3)

## Fit grid search to training data and save it as a grid of results
#rf_grid_results = rf_grid_search.fit(X_train, y_train_rav)

## Determine the best model (the one that works best on left-out obs in cross validation)
#best_model_rf = rf_grid_results.best_estimator_

## Explicate the parameters that make up the best estimator
#best_parameters = rf_grid_results.best_params_

## Identify the best results (Mean cross-validated score of the best_estimator)
#best_results = rf_grid_results.best_score_

## Let's see what we're working with
#print("Best parameters are", best_parameters)
#print("Best parameters produce mean test score of", best_results)

The best hyperparameters are {'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 1e-07, 'n_estimators': 150, 'random_state': 10}

This estimator's mean test score = 0.9269269708905233

In [ ]:
# THE TUNED MODEL
# Here, we hard-code the best_model_rf so that we don't have to re-run the time-intensive tuning code
best_estimator_rf = RandomForestClassifier(n_estimators=150, 
                                           max_depth=10,
                                           max_features=None,
                                           min_impurity_decrease=1e-07,
                                           random_state=10)

best_model_rf = best_estimator_rf.fit(X_train, y_train_rav)

In [ ]:
# With the parameters tested out, the best estimator decreased accuracy on the training set.
# Let's see where this error is coming from by using a confusion matrix.
best_y_train_pred = best_model_rf.predict(X_train)

# create a confusion matrix to allow us to evaluate the tuned classification model on the training set
best_model_rf_train = confusion_matrix(y_train_rav, best_y_train_pred, normalize = "true")

best_rf_train_cm = pd.DataFrame(best_model_rf_train, range(2), range(2))

best_rf_train_cm = best_rf_train_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
best_rf_train_cm.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4) #for label size
sns.heatmap(best_rf_train_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix for Tuned Model on Training Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

It appears that the tuned model `best_model_rf` performs slightly worse (True Pass = 0.95, True Fail = 0.98) than the baseline model `rf_model` (True Pass = 0.99, True Fail = 0.99) on the *training* set.

However, what we lost in training set accuracy, we may have gained in *validation set* accuracy. Let's evaluate it now on the validation set.

In [ ]:
# All the above was done using the training set. 
# Now let's use this best model created with the gridsearch to predict on the validation set
best_y_val_pred = best_model_rf.predict(X_validate)

# create a confusion matrix to allow us to evaluate the tuned classification model on the validation set
best_model_rf_cf = confusion_matrix(y_validate_rav, best_y_val_pred, normalize = "true")

best_rf_cm = pd.DataFrame(best_model_rf_cf, range(2), range(2))

best_rf_cm = best_rf_cm.rename(index=str, columns={0: "Pass", 1: "Fail"})
best_rf_cm.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4) #for label size
sns.heatmap(best_rf_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix for Tuned Model on Validation Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

The tuned model performed better than the baseline model, with 91% of Fails accurately predicted as such, compared with 89% of Fails accurately before, with the baseline model. The proportion of Passes accurately predicted as Passes dropped only a miniscule amount, from 93.8% to 93.2%. `best_model_rf` is superior to `rf_model`

Another way that we might restrict overfitting to the training data and promote accuracy on the validation set would be to reduce the features considered. Let's take a look at what the top features are.

In [ ]:
# Let's look to what factors are most important.
feat_importances = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(rf_model.feature_importances_))], axis = 1)
feat_importances.columns = ["Feature", "Importance"]
sns.barplot(x = "Importance", y = "Feature", data = feat_importances.nlargest(20, 'Importance'))
plt.show()

It looks like only the top fifteen contribute much at all. The `seriousCount` seems to have a disproportionate effect on the model. Let's reduce the features included, removing seriousCount, and see if that helps us create more fine-grained results.

In [ ]:
#X_train_restricted = X_train.drop(columns='seriousCount')
#X_validate_restricted = X_validate.drop(columns='seriousCount')

X_noSC = X.drop(columns="seriousCount")
# split the reduced data into training and validation set
X_train_restricted, X_validate_restricted, y_train_restricted, y_validate_restricted = train_test_split(X_noSC, y, train_size=0.80, test_size=0.20)

In [ ]:
# Let's check if y_train_restricted and y_validate_restricted are in the right shape or need to be raveled.
print(y_train_restricted.shape, y_validate_restricted.shape)

In [ ]:
# Let's fix the shape of the y sets
y_train_restricted = y_train_restricted.ravel()
y_validate_restricted = y_validate_restricted.ravel()

In [ ]:
# Since we're fitting on a different set of features, we need to re-do the hyperparameter tuning

# Below is the code we WOULD run if we had the computer power

## Define the hyperparameters to be tested out
## Options include the default and an alternate option
#param_grid_rf_restricted = {
#    'n_estimators': [100, 150],
#    'max_depth': [10, None],
#    'max_features': [None, 'sqrt'],
#    'max_leaf_nodes': [100, 200],
#    'min_impurity_decrease': [0.0000001, 0.0],
#    'random_state': [10]
#}

## Initiate grid search
#rf_grid_search_restricted = GridSearchCV(rf_classifier, param_grid_rf_restricted, cv=3)

## Fit grid search to training data and save it as a grid of results
#rf_grid_results_restricted = rf_grid_search_restricted.fit(X_train_restricted, y_train_restricted)

## Determine the best model (the one that works best on left-out obs in cross validation)
#best_model_rf_restricted = rf_grid_results_restricted.best_estimator_

## Explicate the parameters that make up the best estimator
#best_parameters_restricted = rf_grid_results_restricted.best_params_

## Identify the best results (Mean cross-validated score of the best_estimator)
#best_results_restricted = rf_grid_results_restricted.best_score_

## Let's see what we're working with
#print("Best parameters are", best_parameters_restricted)
#print("Best parameters produce mean test score of", best_results_restricted)

In [ ]:
# However, we don't have the necessary computer power or time, so instead we'll fit a model using the hyperparameters
# selected by the previous grid search, AKA best_model_rf

best_model_rf_restricted = best_estimator_rf.fit(X_train_restricted, y_train_restricted)

# Then, we make a prediction using this model and evaluate how it does on the validation set.

# Predictions using the "tuned" model fitted to the data excluding "seriousCount"
best_y_val_pred_restricted = best_model_rf_restricted.predict(X_validate_restricted)

# create a confusion matrix to allow us to evaluate the tuned classification model on the validation set
best_model_rf_cf_restricted = confusion_matrix(y_validate_restricted, best_y_val_pred_restricted, normalize = "true")

best_rf_cm_restricted = pd.DataFrame(best_model_rf_cf_restricted, range(2), range(2))

best_rf_cm_restricted = best_rf_cm_restricted.rename(index=str, columns={0: "Pass", 1: "Fail"})
best_rf_cm_restricted.index = ["Pass", "Fail"]
plt.figure(figsize = (8,5))
sns.set(font_scale=1.4) #for label size
sns.heatmap(best_rf_cm_restricted, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix for Restricted Tuned Model on Validation Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

**Conclusion:** The best Random Forests model to use is `best_model_rf` which is fit to all the features in `X_train` and `y_train.ravel()` (AKA `y_train_rav`) and uses the following hyperparameters:

* {'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 1e-07, 'n_estimators': 150, 'random_state': 10}

All hyperparameters not mentioned use the default hyperparameters provided by `sklearn.ensemble.RandomForestClassifier`.

### 3.3.3  - Random Forests Classifier Validation Metrics

In this section, accuracy, precision, recall, F1 score, and cross_val_score will be determined for the Random Forests classifier on the validation set.

In [ ]:
X_validate.shape

In [ ]:
y_validate_rav.shape

In [ ]:
# Create the predictions for the validation set.

y_validate_pred = best_model_rf.predict(X_validate)

<font color = "purple"> I don't know why this problem is happening. Halp.
    -Nadia </font>

### Accuracy
A measure of the number of correction predictions regardless of direction, divided by the total number observations. Accuracy can be expressed as:

$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(y_validate_pred)): 
    if y_validate[i]==y_validate_pred[i]==1:
       TP += 1
    if y_validate_pred[i]==1 and y_validate[i]!=y_validate_pred[i]:
       FP += 1
    if y_validate[i]==y_validate_pred[i]==0:
       TN += 1
    if y_validate_pred[i]==0 and y_validate_pred[i]!=y_validate[i]:
       FN += 1

accuracy = (TP + TN)/(TP + TN + FP + FN)
print("Accuracy of the Random Forests model is", accuracy)

Random Forest's accuracy (=0.927) is a slight improvement over the Decision Tree classifier's accuracy (=0.914).

### Precision
Precision is a measure of how well calibrated predictions are. Specifically, it tells us of the predictions in the positive class ("failing inspection" in this case) we made, how many were correct. The formula for precision is:

$$
Precision = \frac{TP}{TP + FP}
$$

In [ ]:
precision = TP/(TP + FP)
print("Precision of the Random Forests mdoel is", precision)

The Decision Tree classifier's precision (=0.781) is slightly better than Random Forests precision (=0.771).

### Recall
Recall is a measure that tells us, of all of the positive class members in the ground truth labels, how many did we successfully predict as positive? Recall is defined as:

$$
Recall = \frac{TP}{TP + FN}
$$

In [ ]:
recall = TP/(TP+FN)
print("Recall of the Random Forests mdoel is", recall)

The Random Forests model performs *significantly* better (recall=0.910) than the Decision Tree model (recall=0.795) in this measure.

### F1 Score
The F1 Score is defined as:

$$
F1 = 2 * \frac{precision * recall}{precision + recall}
$$


In [ ]:
f1 = 2 * (precision * recall)/(precision + recall)
print("F1 Score for Random Forests is", f1)

The Random Forests f1 score (=0.835) is moderately better than that of the Decision Tree model (=0.788).

### Cross-Validation Score

In [ ]:
scores = cross_val_score(rf_model, X_validate, y_validate, cv=5)
print(scores.mean())

In [ ]:
scores = cross_val_score(rf_model, X, y.ravel(), cv=5)
print(scores.mean())

The mean cross validation scores for the Random Forests model, both when run on the validation set alone (mean cross val score = 0.919) and on the full set (mean cross val score = 0.922), are higher than the full-set mean cross_val_score of the Decision Tree model (=0.905) and the reduced Decision Tree model (=0.892).

# 4.  Policy Simulation

## 4.1  Interpretable Machine Learning

**Hint**: Use tools like feature importance plots and coefficient plots

### 4.1.1 Model 1 - SVM Feature Importance Plot

In [ ]:
from sklearn import datasets, svm
from sklearn.feature_selection import SelectPercentile, f_classif

In [ ]:
X_indices = np.arange(X.shape[-1])
# Univariate feature selection with F-test for feature scoring
# We use the default selection function: the 10% most significant features
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(X, y.ravel())
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
plt.bar(X_indices - .45, scores, width=.2,
        label=r'Univariate score ($-Log(p_{value})$)', color='g')

# Compare to the weights of an SVM
clf = svm.SVC(kernel='linear')
clf.fit(X, y.ravel())

svm_weights = (clf.coef_ ** 2).sum(axis=0)
svm_weights /= svm_weights.max()

plt.bar(X_indices - .25, svm_weights, width=.2, label='SVM weight', color='r')

clf_selected = svm.SVC(kernel='linear')
clf_selected.fit(selector.transform(X), y.ravel())

svm_weights_selected = (clf_selected.coef_ ** 2).sum(axis=0)
svm_weights_selected /= svm_weights_selected.max()

plt.bar(X_indices[selector.get_support()] - .05, svm_weights_selected,
        width=.2, label='SVM weights after selection', color='b')

plt.title("Comparing feature selection")
plt.xlabel('Feature number')
plt.yticks(())
plt.axis('tight')
plt.legend(loc='upper right')
plt.show()

# got an error that SVC' object has no attribute 'feature_importances_' if using code from the lab
# the above code is from https://www.cnpython.com/qa/321322
# but not able to run through 

In [ ]:
## create a dataframe with coefficients and feature names
svm_data = pd.DataFrame([svm_model.coef_, 
                             chicago_inspections_2011_to_2013.columns]).T

## Output gets error message: "coef_ is only available when using a linear kernel"

### 4.1.2 Model 2 - Decision Tree Classifier Feature Importance Plot

In [ ]:
# Replicating Feature Importance Plot from 3.2.2 above
dt_classifier.fit(X_train, y_train)
feat_importances = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(dt_classifier.feature_importances_))], axis = 1)
feat_importances.columns = ["Feature", "Importance"]
sns.barplot(x = "Importance", y = "Feature", data = feat_importances.nlargest(10, 'Importance'))
plt.show()

#### Interpretation: 
We see that seriousCount is by far the most important feature in the model, far more important than the the next 9 most important features combined. One feature that I wish we could incorporate into the model is which inspector from the Chicago Department of Public Health’s Food Protection Program completed past inpections. Although the inspections are done using a standardized procedure, I wonder if there are were inspectors that were systematically harsher or more lenient and how that might bias the results and subsequent predictions. 

## 4.2  Prioritize Audits

**Hint**: Look up the [`.predict()`](https://www.kite.com/python/docs/sklearn.linear_model.SGDRegressor.predict), [`.predict_proba()`](https://www.kite.com/python/docs/sklearn.linear_model.LogisticRegression.predict_proba), and [`.sample()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) methods. Then: 
1. Choose one of your models (or train a new simplified model or ensemble!) to predict outcomes and probabilities. 
2. Order your audits by their probability of detecting a "Fail" score
3. Plot your distribution of pass/fail among the first 1000 observations in the dataset
4. Simulate random audits on the full chicago_2011_to_2013.csv dataset by picking 1000 observations at random

Let's use Random Forests to determine which facilities to audit.

Random Forests can use `predict_proba()`. In the parentheses we put the X array and then it returns an array where the second column is the proportion of decision trees within the random forest that classify that observation as 1 (signifying "Fail"). From there, we can select out the top 1000 observations with the highest probabilities (i.e., ratios of decision trees classifying them as 1) using `nlargest(1000)`.

In [ ]:
# Let's set it up using the Random Forest model of our choice
# Ensure that the model/estimator variable preceding .predict_proba() is the best model for the job
prob_pred_rf_val = best_model_rf.predict_proba(X_validate)

# prob_pred_rf_val is an array where the second column = predicted probabilities of Failed inspection

# Let's print the array in descending order, sorted by the second number in each sub-list
print(prob_pred_rf_val.sort(key=sortSecond,reverse=True))

<font color = "purple"> I am following an example I saw so I don't know what I'm doing wrong. Suggestions appreciated.
    -Nadia </font>

In [ ]:
# Trying to figure out how to select the 1000 riskiest establishments
X['Risk_Risk 1 (High)'].sum()

In [ ]:
chicago_inspections_2011_to_2013['seriousCount'].unique()

In [ ]:
# We could try to taking a random sample of the establishments with the highest risk of 1 who also have a serious count >1? 
chicago_inspections_2011_to_2013['seriousCount'].value_counts()

In [ ]:
X[('Risk_Risk 1 (High)'== 1) & ('seriousCount'>1)].count()

In [ ]:
# Do we need to change one of the dtypes to be able to run the above code?
X.dtypes['Risk_Risk 1 (High)']

In [ ]:
X.dtypes['seriousCount']

## 4.3  Predict on Data with Unseen Labels

In [ ]:
# Fill in the code below with the X data you used for training
X_test = chicago_inspections_2014[chicago_inspections_2014.columns & ....columns]

# 5. Discussion Questions

### 5.1 Why do we need metrics beyond accuracy when using machine learning in the social sciences and public policy?

We need metrics beyond accuracy when using machine learning in the social sciences and public policy because often times the direction of our prediction, that is, positive or negative class in this case, often has real-world implications that may cause people and communities harm. In our case, predicting a passing inspection (TN or FN) is particularly consequential. To the extent that we incorrectly predict passing inspections for food establishments, we may be putting members of the community in harm's way by enabling businesses that should not be operating to serve them unhealthy food. Thus, in addition to the overall accuracy of our predictive model, we would be particularly interested in the precision of our positive and negative classifications. This would likely be the case for many other problems in the social sciences and public policy.

Anything to add?

### 5.2 Imagine that establishments learned about the algorithm being used to determine who gets audited and they started adjusting their behavior (and changing certain key features about themselves that were important for the prediction) to avoid detection. How could policymakers address this interplay between algorithmic decisionmaking and real world behavior?

Hmmm....I may ask KQ for some guidance on this question.